In [65]:
import os

from datetime import timezone, datetime

from crypto_trades_downloader.timescaledb_util import TimeScaleDBUtil
import pandas as pd

pd.options.display.precision = 10

# PostgreSQL設定
pg_config = {
    'user': os.environ['POSTGRES_USER'],
    'password': os.environ['POSTGRES_PASSWORD'],
    'host': os.environ['POSTGRES_HOST'],
    'port': os.environ['POSTGRES_PORT'],
    'database': os.environ['POSTGRES_DATABASE']
}

dbutil = TimeScaleDBUtil(user = pg_config['user'], password = pg_config['password'], host = pg_config['host'], port = pg_config['port'], database = pg_config['database'])

In [76]:
ex_a_name = 'ftx'
ex_b_name = 'poloniex'

table_name = dbutil.get_trade_table_name(ex_a_name, 'XRP/BTC')
df_trades_a = dbutil.read_sql_query(sql = f'SELECT * FROM "{table_name}" WHERE datetime < \'2021-12-26 00:00:00+00\' ORDER BY datetime')
df_trades_a = df_trades_a.rename(columns={'price': 'price_a'})
df_trades_a['exchange'] = ex_a_name

table_name = dbutil.get_trade_table_name(ex_b_name, 'XRP/BTC')
df_trades_b = dbutil.read_sql_query(sql = f'SELECT * FROM "{table_name}" WHERE datetime < \'2021-12-26 00:00:00+00\' ORDER BY datetime')
df_trades_b = df_trades_b.rename(columns={'price': 'price_b'})
df_trades_b['exchange'] = ex_b_name

In [86]:
df_ab_trades_nona = pd.concat([df_trades_a, df_trades_b]).sort_values('datetime', ascending=True).fillna(method='ffill').dropna().reset_index(drop=True)
df_ab_trades_nona['price_diff'] = df_ab_trades_nona['price_a'] - df_ab_trades_nona['price_b']
df_ab_trades_nona['price_diff_pct'] = df_ab_trades_nona['price_diff'] / df_ab_trades_nona['price_a'] * 100
df_ab_trades_nona['profitable'] = False
df_ab_trades_nona.loc[(df_ab_trades_nona['price_diff_pct'] > 1.0) | (df_ab_trades_nona['price_diff_pct'] < -1.0), 'profitable'] = True
df_ab_trades_nona['inverse_profitable'] = ~df_ab_trades_nona['profitable']
df_ab_trades_nona['profitable_label'] = df_ab_trades_nona['inverse_profitable'].cumsum()

In [88]:
df_ab_trades_nona_timediff = df_ab_trades_nona.sort_values('datetime', ascending=False)
df_ab_trades_nona_timediff['datetime_diff'] = -df_ab_trades_nona_timediff['datetime'].diff().dt.total_seconds()
df_ab_trades_nona_timediff.dropna(inplace=True)
df_ab_trades_nona_timediff = df_ab_trades_nona_timediff.sort_values('datetime', ascending=True).reset_index(drop=True)

In [89]:
df_ab_trades_nona_timediff_profitable = df_ab_trades_nona_timediff[df_ab_trades_nona_timediff['profitable'] == True]
df_ab_trades_nona_timediff_profitable_group = df_ab_trades_nona_timediff_profitable.groupby("profitable_label")

In [96]:
df_ab_trades_nona_timediff_profitable['datetime_diff'].sum() / 60 / 60

129.26795027777777

In [101]:
from scipy import stats
import numpy as np

for x in range(0, 101, 1):
    percentile = stats.scoreatpercentile(df_ab_trades_nona['price_diff_pct'].abs(), x)
    print(f"{x} percentile: {percentile:.3f}%")

0 percentile: 0.000%
1 percentile: 0.000%
2 percentile: 0.000%
3 percentile: 0.000%
4 percentile: 0.000%
5 percentile: 0.000%
6 percentile: 0.000%
7 percentile: 0.000%
8 percentile: 0.000%
9 percentile: 0.000%
10 percentile: 0.000%
11 percentile: 0.000%
12 percentile: 0.000%
13 percentile: 0.000%
14 percentile: 0.000%
15 percentile: 0.000%
16 percentile: 0.000%
17 percentile: 0.000%
18 percentile: 0.000%
19 percentile: 0.037%
20 percentile: 0.038%
21 percentile: 0.039%
22 percentile: 0.040%
23 percentile: 0.042%
24 percentile: 0.042%
25 percentile: 0.044%
26 percentile: 0.045%
27 percentile: 0.046%
28 percentile: 0.048%
29 percentile: 0.051%
30 percentile: 0.052%
31 percentile: 0.052%
32 percentile: 0.053%
33 percentile: 0.054%
34 percentile: 0.054%
35 percentile: 0.055%
36 percentile: 0.055%
37 percentile: 0.056%
38 percentile: 0.057%
39 percentile: 0.057%
40 percentile: 0.058%
41 percentile: 0.060%
42 percentile: 0.066%
43 percentile: 0.075%
44 percentile: 0.078%
45 percentile: 0.081

In [97]:
from scipy import stats
import numpy as np

df_time_diff = df_ab_trades_nona_timediff_profitable_group['datetime_diff'].sum()

for x in range(0, 105, 5):
    percentile = stats.scoreatpercentile(df_time_diff, x)
    print(f"{x} percentile: {percentile:.2f} sec")

0 percentile: 0.00 sec
5 percentile: 0.00 sec
10 percentile: 0.00 sec
15 percentile: 0.00 sec
20 percentile: 0.00 sec
25 percentile: 0.00 sec
30 percentile: 0.30 sec
35 percentile: 0.88 sec
40 percentile: 1.00 sec
45 percentile: 1.26 sec
50 percentile: 2.00 sec
55 percentile: 3.00 sec
60 percentile: 4.00 sec
65 percentile: 6.10 sec
70 percentile: 10.00 sec
75 percentile: 15.00 sec
80 percentile: 24.00 sec
85 percentile: 40.00 sec
90 percentile: 71.00 sec
95 percentile: 155.29 sec
100 percentile: 5665.71 sec


In [95]:
len(df_ab_trades_nona_timediff_profitable)

118448